# Датасет с ирисами: классификация и градиентный спуск

Загружаю датасет

In [3]:
from sklearn.datasets import load_iris
import pandas as pd

# Загружаем датасет
iris = load_iris()

# Сразу создаем DataFrame и фильтруем
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Оставляем только классы 1 (Versicolor) и 2 (Virginica)
filtered_df = iris_df[iris_df['target'] >= 1].copy()

# Для ясности добавляем названия классов
class_names = {1: 'versicolor', 2: 'virginica'}
filtered_df['target_name'] = filtered_df['target'].map(class_names)

print("Отфильтрованный датасет (2 класса):")
print(f"Размер: {len(filtered_df)} записей")
print(f"\nРаспределение классов:")
print(filtered_df['target_name'].value_counts())
print(f"\nПропорции: {filtered_df['target_name'].value_counts(normalize=True).round(2).to_dict()}")

Отфильтрованный датасет (2 класса):
Размер: 100 записей

Распределение классов:
target_name
versicolor    50
virginica     50
Name: count, dtype: int64

Пропорции: {'versicolor': 0.5, 'virginica': 0.5}


Логистическая регрессия

In [4]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            # Прямое распространение
            linear = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear)

            # Градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Обновление параметров
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_proba(self, X):
        linear = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

Использование

In [5]:
# Пример с данными ирисов
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Загружаем и фильтруем данные
iris = load_iris()
X = iris.data[iris.target != 0]
y = iris.target[iris.target != 0]
y = y - 1  # 0 и 1 вместо 1 и 2

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и обучаем модель
model = LogisticRegression(learning_rate=0.1, n_iterations=1000)
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

# Точность
accuracy = np.mean(y_pred == y_test)
print(f"Точность: {accuracy:.4f}")

Точность: 0.8500


Метод градиентного спуска

In [6]:
import numpy as np

class LogisticRegressionGD:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iterations = n_iterations
        self.w = None
        self.b = None
        self.loss_history = []

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def _compute_loss(self, y_true, y_pred):
        # Log loss (binary cross entropy)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        self.loss_history = []

        # Градиентный спуск
        for i in range(self.n_iterations):
            # Прямое распространение
            linear = X @ self.w + self.b
            y_pred = self._sigmoid(linear)

            # Вычисление потерь
            loss = self._compute_loss(y, y_pred)
            self.loss_history.append(loss)

            # Градиенты
            dw = (1 / n_samples) * (X.T @ (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Обновление параметров
            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict_proba(self, X):
        linear = X @ self.w + self.b
        return self._sigmoid(linear)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

# Метрика качества - точность (accuracy)
def calculate_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Пример использования
if __name__ == "__main__":
    # Данные ирисов (Versicolor и Virginica)
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Загрузка и подготовка данных
    iris = load_iris()
    X = iris.data[iris.target != 0]  # Только 2 класса
    y = iris.target[iris.target != 0]
    y = y - 1  # Преобразуем в 0 и 1

    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Создание и обучение модели
    model = LogisticRegressionGD(learning_rate=0.1, n_iterations=1000)
    model.fit(X_train_scaled, y_train)

    # Предсказания
    y_pred = model.predict(X_test_scaled)

    # Расчет метрики
    accuracy = calculate_accuracy(y_test, y_pred)

    # Вывод результатов
    print(f"Точность (accuracy) на тестовой выборке: {accuracy:.4f}")
    print(f"Финальная loss: {model.loss_history[-1]:.6f}")

Точность (accuracy) на тестовой выборке: 0.8000
Финальная loss: 0.050086


Метод скользящего среднего (Root Mean Square Propagation, RMSProp)

In [7]:
import numpy as np

class LogisticRegressionRMSProp:
    def __init__(self, learning_rate=0.01, n_iterations=1000, beta=0.9, epsilon=1e-8):
        self.lr = learning_rate
        self.n_iterations = n_iterations
        self.beta = beta
        self.epsilon = epsilon
        self.w = None
        self.b = None
        self.loss_history = []

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def _compute_loss(self, y_true, y_pred):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        self.loss_history = []

        # Инициализация для RMSProp
        s_w = np.zeros(n_features)  # для весов
        s_b = 0                     # для смещения

        # Градиентный спуск с RMSProp
        for i in range(self.n_iterations):
            # Прямое распространение
            linear = X @ self.w + self.b
            y_pred = self._sigmoid(linear)

            # Вычисление потерь
            loss = self._compute_loss(y, y_pred)
            self.loss_history.append(loss)

            # Градиенты
            dw = (1 / n_samples) * (X.T @ (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # RMSProp: обновление скользящих средних квадратов градиентов
            s_w = self.beta * s_w + (1 - self.beta) * (dw ** 2)
            s_b = self.beta * s_b + (1 - self.beta) * (db ** 2)

            # Обновление параметров с адаптивным шагом
            self.w -= self.lr * dw / (np.sqrt(s_w) + self.epsilon)
            self.b -= self.lr * db / (np.sqrt(s_b) + self.epsilon)

    def predict_proba(self, X):
        linear = X @ self.w + self.b
        return self._sigmoid(linear)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

# Метрика качества - точность (accuracy)
def calculate_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Пример использования
if __name__ == "__main__":
    # Данные ирисов (Versicolor и Virginica)
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Загрузка и подготовка данных
    iris = load_iris()
    X = iris.data[iris.target != 0]
    y = iris.target[iris.target != 0]
    y = y - 1

    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Создание и обучение модели с RMSProp
    model = LogisticRegressionRMSProp(learning_rate=0.01, n_iterations=1000, beta=0.9)
    model.fit(X_train_scaled, y_train)

    # Предсказания
    y_pred = model.predict(X_test_scaled)

    # Расчет метрики
    accuracy = calculate_accuracy(y_test, y_pred)

    # Вывод результатов
    print(f"Точность (accuracy) на тестовой выборке: {accuracy:.4f}")
    print(f"Финальная loss: {model.loss_history[-1]:.6f}")

Точность (accuracy) на тестовой выборке: 0.8000
Финальная loss: 0.009426
